In [1]:
import pandas as pd
import json
import datasets

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
model_checkpoint = "bert-base-multilingual-cased"
batch_size = 16

In [4]:
with open('data/zac2022_train_merged_final.json',encoding='utf-8') as f:
    data=json.load(f)
    df = pd.json_normalize(data,'data')

# 2 trường hợp output là có hoặc không có câu trả lời
df = df[(df['category'] == 'FULL_ANNOTATION') | (df['category'] == 'FALSE_LONG_ANSWER')]
# độ dài của câu trả lời
df['short_candidate_length'] = df['short_candidate'].apply(lambda x: len(x) if type(x) == str else 0)
# remove col 
df = df.drop(['answer','title','is_long_answer','category','id'], axis=1)
print(len(df))
df.head()

17359


,question,text,short_candidate_start,short_candidate,short_candidate_length
1,Đất nước nào không có quân đội,"có 23 quốc gia không có lực lượng quân đội, ba...",53.0,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",73
2,Pháp tấn công xâm lược Việt Nam vào ngày tháng...,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,NaN,NaN,0
3,Cờ vua còn có tên gọi nào khác,Cúp cờ vua thế giới là tên gọi một số giải đấu...,NaN,NaN,0
4,Núi nào cao nhất châu âu,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,73.0,núi Elbrus,10
5,Những cây thánh giá tại ngọn đồi Thánh giá ở L...,Ngọn đồi Thánh giá (tiếng Litva: ) là địa điểm...,NaN,NaN,0


In [5]:
mlqa_df = pd.read_csv('data/MLQA_V1_vi.csv')
# rename column
mlqa_df.rename(columns={'context':'text','answer':'short_candidate','answer_start':'short_candidate_start'}, inplace=True)
mlqa_df['short_candidate_length'] = mlqa_df['short_candidate'].apply(lambda x: len(x) if type(x) == str else 0)
print(len(mlqa_df))
mlqa_df.head()

6006


,text,question,short_candidate,short_candidate_start,title,short_candidate_length
0,Khí hậu khác biệt theo từng vùng. Thung lũng F...,Bao nhiêu ngày mùa đông dưới 0 độ?,40 ngày,341,Kyrgyzstan,7
1,Kyrgyzstan là một trong hai trong số năm nước ...,Trước đây bảng chữ cái Kyrgyz được viết bằng n...,ký tự Ả Rập,510,Kyrgyzstan,11
2,"Nói chung, tất cả mọi người trong nước đều hiể...",Phiên dịch được sử dụng cho ngôn ngữ nào?,Kyrgyz,276,Kyrgyzstan,6
3,Vận tải Kyrgyzstan ở tình trạng kém phát triển...,Các con đường ở Kyrgyzstan thường chịu ảnh hưở...,thường bị lở đất hay lở tuyết.,220,Kyrgyzstan,30
4,Đại điện nhà trai là Hoàng hậu Elisabeth và Th...,Các vị thế của tòa án Berlin đối với Anh và Ng...,chống Anh và thân Nga,993,"Viktoria, Hoàng hậu Đức",21


In [6]:
df = pd.concat([df, mlqa_df], ignore_index=True)

In [7]:
# format theo code copy :v
dataset = datasets.Dataset.from_pandas(df)
dataset = dataset.map(lambda example: {'answers': {'answer_start': [example['short_candidate_start']] if example['short_candidate_start']!=None else [],
                                                           'text': [example['short_candidate']] if example['short_candidate']!=None else[]}})
dataset = dataset.map(lambda example: {'context': example['text']}, batched=True, remove_columns='text')
# drop columns that are not needed
dataset = dataset.remove_columns(['short_candidate_start','title', 'short_candidate', 'short_candidate_length'])
# split 
dataset = dataset.train_test_split(test_size=0.1,seed=42)

  0%|          | 0/23365 [00:00<?, ?ex/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [8]:
dataset['train'][0]

{'question': 'Các dịch gì thường đi kèm với những triệu chứng này?',
 'answers': {'answer_start': [219.0], 'text': ['khí hư trắng']},
 'context': 'Nhiễm nấm âm đạo, là sự phát triển quá nhiều của nấm men trong âm đạo đến mức gây kích thích. Các triệu chứng phổ biến nhất là ngứa âm đạo, có thể đến mức nghiêm trọng. Các triệu chứng khác bao gồm đau rát khi đi tiểu, khí hư trắng và dày, thường không có mùi, đau khi quan hệ tình dục và đỏ tấy xung quanh âm đạo. Triệu chứng thường trở nên xấu đi trước kỳ kinh nguyệt của phụ nữ.Nhiễm nấm âm đạo là do sự phát triển quá nhiều của nấm Candida. Nấm men này thường xuất hiện trong âm đạo với số lượng nhỏ. Bệnh này không được phân loại như một bệnh truyền nhiễm qua đường tình dục, tuy nhiên, nó có thể xảy ra thường xuyên hơn đối với những phụ nữ thường xuyên hoạt động tình dục. Các yếu tố rủi ro bao gồm việc dùng thuốc kháng sinh, mang thai, bệnh tiểu đường, và HIV/AIDS. Chế độ ăn đường đơn cũng có thể gây nấm. Quần áo chật, loại đồ lót, vệ sinh cá

In [9]:
import transformers
from transformers import AutoTokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
max_length = 384
doc_stride = 128
pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [12]:
features = prepare_train_features(dataset["train"][:5])

In [13]:
tokenized_datasets = dataset.map(
    prepare_train_features, batched=True, remove_columns=dataset["train"].column_names
)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [14]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [15]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"model_2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
)

In [16]:
from transformers import default_data_collator

data_collator = default_data_collator

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    
)

In [18]:
trainer.train()

d:\Software\Anaconda\envs\rs\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22944
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 71700
  Number of trainable parameters = 177264386


  0%|          | 0/71700 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.save_model("model_2/saved")

In [ ]:
# test model
from transformers import pipeline


qa = pipeline('question-answering', model='model_2\checkpoint-28391',device=0,)

In [ ]:
with open('data/zac2022_train_merged_final.json',encoding='utf-8') as f:
    data=json.load(f)
    df = pd.json_normalize(data,'data')

In [ ]:
# PARTIAL_ANNOTATION
for c in ['PARTIAL_ANNOTATION','FULL_ANNOTATION','FALSE_LONG_ANSWER']:
    print(c)
    example = df[df['category'] == c].sample(1)
    question = example['question'].values[0]
    context = example['text'].values[0]
    print('QUESTION:', question)
    print('CONTEXT:', context)
    print('is long answer:',example['is_long_answer'].values[0])
    print(qa(question=question, context=context))
    print()

PARTIAL_ANNOTATION
QUESTION: Thiên đường thuế là gì
CONTEXT: 1. không đánh thuế hoặc thuế gần như bằng không. Thiên đường thuế biến nó thành nơi để những người không cư trú ở đó tránh khỏi phải đóng thuế cao ở nơi họ ở hay kinh doanh.
is long answer: True
{'score': 7.756276562104565e-18, 'start': 49, 'end': 65, 'answer': 'Thiên đường thuế'}

FULL_ANNOTATION
QUESTION: thái bình là một thành phố trực thuộc tỉnh nào
CONTEXT: Ngày 12 tháng 12 năm 2013 , Thủ tướng Chính phủ ra Quyết định 2418 / QĐ-TTg công nhận Thành phố Thái Bình là đô thị loại II trực thuộc tỉnh Thái Bình .
is long answer: True
{'score': 0.9999990463256836, 'start': 135, 'end': 151, 'answer': 'tỉnh Thái Bình .'}

FALSE_LONG_ANSWER
QUESTION: Quốc gia nào là quốc gia đầu tiên đoạt chức vô địch Worldcup
CONTEXT: Năm 1971, Franz Beckenbauer trở thành đội trưởng của đội tuyển quốc gia. Ông đã dẫn dắt đội tuyển đoạt chức vô địch châu Âu Euro 1972, khi đánh bại Liên Xô 3–0 trong trận chung kết.
is long answer: False
{'score': 6.

In [ ]:
question = 'nhà chiêm tinh thiên văn học nổi tiếng tycho brahe là người nước nào'
context = 'B. - Tycho Brahe ( 1546-1601 ) , nhà thiên văn học người Đan Mạch . - Brahmagupta ( 598-668 ) , nhà thiên văn học người Ấn Độ .'
qa(question=question, context=context)